## Урок 9. Интеграция. Итоговый проект
https://www.kaggle.com/sulianova/cardiovascular-disease-dataset?select=cardio_train.csv

**Описание данных:**<br/>
- age- возраст в днях;<br/>
- height- рост в см;<br/>
- weigh - вес в кг;<br/>
- gende - пол;<br/>
- ap_hi - систолическое артериальное давление;<br/>
- ap_lo - диастолическое артериальное давление;<br/>
- cholesterol - холестерин (1: нормальный, 2: выше нормы, 3: значительно выше нормы);<br/>
- gluc - глюкоза (1: нормальная, 2: выше нормы, 3: значительно выше нормы);<br/>
- smoke - курение;<br/>
- alco - употребление алкоголя;<br/>
- active - физическая активность;<br/>
- cardio - наличие или отсутствие сердечно-сосудистых заболеваний.<br/>

Загружу и проанализирую данные

In [1]:
#!pip install xgboost
#!pip install dill

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import dill

from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
df = pd.read_csv('cardio_train.csv', sep=";")
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [4]:
first_len = len(df)
first_len

70000

In [5]:
df.drop("id",axis=1,inplace=True)
print(f'в загруженном дата сэте {df.duplicated().sum()} записи-дубликата')

в загруженном дата сэте 24 записи-дубликата


In [6]:
duplicated = df[df.duplicated(keep=False)]
duplicated = duplicated.sort_values(by=['age', "gender", "height"], ascending= False) 
duplicated.head(4)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
2677,22077,1,175,69.0,120,80,1,1,0,0,1,1
45748,22077,1,175,69.0,120,80,1,1,0,0,1,1
1568,21945,1,165,60.0,120,80,1,1,0,0,1,0
48917,21945,1,165,60.0,120,80,1,1,0,0,1,0


In [7]:
df.drop_duplicates(inplace=True)
print(f'в загруженном дата сэте {df.duplicated().sum()} записи-дубликата')

в загруженном дата сэте 0 записи-дубликата


In [8]:
last_len = len(df)
print(f'В дата сэте {last_len} записи(ей)\n\
удалено {first_len-last_len} записи(ей)')

В дата сэте 69976 записи(ей)
удалено 24 записи(ей)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69976 entries, 0 to 69999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          69976 non-null  int64  
 1   gender       69976 non-null  int64  
 2   height       69976 non-null  int64  
 3   weight       69976 non-null  float64
 4   ap_hi        69976 non-null  int64  
 5   ap_lo        69976 non-null  int64  
 6   cholesterol  69976 non-null  int64  
 7   gluc         69976 non-null  int64  
 8   smoke        69976 non-null  int64  
 9   alco         69976 non-null  int64  
 10  active       69976 non-null  int64  
 11  cardio       69976 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.9 MB


In [10]:
df.describe()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,69976.000000,69976.000000,69976.000000,69976.000000,69976.000000,69976.000000,69976.000000,69976.000000,69976.000000,69976.000000,69976.000000,69976.000000
mean,19468.950126,1.349648,164.359152,74.208519,128.820453,96.636261,1.366997,1.226535,0.088159,0.053790,0.803718,0.499771
std,2467.374620,0.476862,8.211218,14.397211,154.037729,188.504581,0.680333,0.572353,0.283528,0.225604,0.397187,0.500004
min,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


Разделю данные на train и test и сохраним тестовую выборку

In [11]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

Добавлю XGBClassifier

In [12]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [13]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [14]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-0.57376123, -1.49787908, -1.05398797, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.58597416, -0.77006046,  0.33554054, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.2826264 , -1.25527287, -1.54032294, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.32763711, -1.01266667,  0.61344624, ...,  0.        ,
         0.        ,  1.        ],
       [-0.48982234, -1.8617884 , -2.09613435, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.10261591,  1.29209233,  0.96082837, ...,  0.        ,
         0.        ,  1.        ]])

In [15]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', xgb.XGBClassifier(use_label_encoder=False)),
])

pipeline.fit(X_train, y_train)

[23:01:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                        

In [16]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('age',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='age')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('height',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='height')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('weight',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='weight')),
                                                  ('standard',
                                                   StandardScaler())])),
                 

In [17]:
with open("XGBClassifier_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)